In [1]:
# Parameters
cell_type = "pDC"


In [2]:
import sys
import os

import anndata as ad

import pandas as pd
import numpy as np

from pyprojroot import here

import re

from tqdm import tqdm

In [3]:
def load_sorted_shap_values_fnames(
    cell_type: str = '',
    run_name: str = 'run1'):

    dirpath = here(f"inflammabucket_bkp/03_downstream_analysis/05_SHAP/results/04_shap/shap_vals")
    fname_regex = f'^{run_name}_{cell_type}_shap_values_' + r'(\d+)'
    results_batches = [
        (fname, int(re.search(fname_regex, fname).group(1)))
        for fname in os.listdir(dirpath)
        if re.search(fname_regex, fname)]

    sorted_files = sorted(results_batches, key=lambda x: x[1])

    sorted_filenames = [os.path.join(dirpath, filename) for filename, _ in sorted_files]

    return sorted_filenames

In [4]:
shap_val_list = [np.load(fname)['shap_values'] for fname in tqdm(load_sorted_shap_values_fnames(cell_type))] 


  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:03<00:07,  3.81s/it]

 67%|██████▋   | 2/3 [00:06<00:03,  3.19s/it]

100%|██████████| 3/3 [00:07<00:00,  2.22s/it]

100%|██████████| 3/3 [00:07<00:00,  2.55s/it]

In [5]:
adata = ad.read_h5ad(here(f'inflammabucket_bkp/03_downstream_analysis/05_SHAP/data/{cell_type}_adataMerged_SPECTRAgenes.log1p.h5ad'), backed='r')

In [6]:
## Gene symbols and Disease labels
symbols_df = pd.read_pickle(here('inflammabucket_bkp/03_downstream_analysis/04_selected_gene_list.pkl'))
symbols_sorted = symbols_df.loc[adata.var_names].symbol.values

DISEASES = ['BRCA', 'CD', 'COPD', 'COVID', 'CRC', 'HBV', 'HIV', 'HNSCC', 'MS', 'NPC', 'PS', 'PSA', 'RA', 'SLE', 'UC', 'asthma', 'cirrhosis', 'flu', 'healthy', 'sepsis']

diseaseDict = dict()
for d in DISEASES:
    diseaseDict[d] = []

In [7]:
shape_values_matrix = np.concatenate(shap_val_list)

In [8]:
for idx, values in tqdm(adata.obs.groupby('sampleID', observed=True).indices.items()):
    geneXdisease_sample_i = pd.DataFrame(shape_values_matrix[values].mean(0))
    geneXdisease_sample_i.columns = DISEASES
    geneXdisease_sample_i.index = symbols_sorted
    for d in geneXdisease_sample_i.columns:
        diseaseDict[d].append(pd.DataFrame.from_dict({idx:geneXdisease_sample_i[d]}))

  0%|          | 0/775 [00:00<?, ?it/s]

  6%|▌         | 45/775 [00:00<00:01, 445.29it/s]

 12%|█▏        | 96/775 [00:00<00:01, 480.07it/s]

 19%|█▉        | 148/775 [00:00<00:01, 495.07it/s]

 26%|██▌       | 198/775 [00:00<00:01, 375.13it/s]

...

 80%|████████  | 621/775 [00:01<00:00, 412.70it/s]

 86%|████████▌ | 664/775 [00:01<00:00, 314.69it/s]

 91%|█████████▏| 709/775 [00:01<00:00, 344.35it/s]

 97%|█████████▋| 753/775 [00:01<00:00, 366.98it/s]

100%|██████████| 775/775 [00:02<00:00, 386.44it/s]

In [9]:
for d in tqdm(DISEASES):
    pd.concat(diseaseDict[d], axis=1).to_csv(here(f'03_downstream_analysis/08_gene_importance/new_shap_plots/results/SHAP_AVGsamples/SHAP_AVGsample_{cell_type}_{d}.csv'))

  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:00<00:10,  1.78it/s]

 10%|█         | 2/20 [00:01<00:14,  1.25it/s]

 15%|█▌        | 3/20 [00:02<00:16,  1.06it/s]

 20%|██        | 4/20 [00:03<00:16,  1.02s/it]

...

 85%|████████▌ | 17/20 [00:17<00:03,  1.29s/it]

 90%|█████████ | 18/20 [00:19<00:02,  1.40s/it]

 95%|█████████▌| 19/20 [00:20<00:01,  1.38s/it]

100%|██████████| 20/20 [00:22<00:00,  1.44s/it]

100%|██████████| 20/20 [00:22<00:00,  1.12s/it]